## Plotting Spotter Drift Trajectories
This notebook demonstrates plotting the latitude, longitude locations during a
 specified time period, colored by significant wave height observed.

Plotly is used to render the geographic data (continents, etc).

In [1]:
# Import necessary packages

import pandas as pd
import plotly.graph_objects as go
import xarray as xr
import datetime


## Load netcdf dataset
xarray is used to load dataset, and convert to a pandas dataframe

In [2]:
# open the file using xarray
filename = "spotter_data_bulk.nc"
bulk_wave_data  = xr.load_dataset(filename)

# print dataset info to see variables and the dimensions (only dimension is the observation index number)
print(bulk_wave_data.info)


<bound method Dataset.info of <xarray.Dataset>
Dimensions:                (index: 91473)
Coordinates:
  * index                  (index) int64 0 1 2 3 4 ... 91469 91470 91471 91472
Data variables:
    spotter_id             (index) object 'SPOT-010001' ... 'SPOT-1972'
    significantWaveHeight  (index) float64 1.547 1.636 1.589 ... 2.536 2.629
    peakPeriod             (index) float64 7.877 8.533 8.533 ... 10.24 11.38
    meanPeriod             (index) float64 6.702 6.716 6.697 ... 8.26 8.401
    peakDirection          (index) float64 99.21 114.2 111.3 ... 71.44 70.04
    peakDirectionalSpread  (index) float64 52.49 44.95 42.0 ... 14.32 13.96 13.2
    meanDirection          (index) float64 100.3 104.9 100.9 ... 65.88 65.23
    meanDirectionalSpread  (index) float64 55.2 53.02 52.5 ... 30.73 33.27 30.62
    time                   (index) float64 1.653e+09 1.653e+09 ... 1.654e+09
    latitude               (index) float64 19.66 19.66 19.66 ... 50.31 50.31
    longitude              (ind

In [3]:
# convert to pandas dataframe for ease of analysis and plotting
bulk_wave_dataframe = bulk_wave_data.to_dataframe()

# convert unix epoch time [s] to datetime (human readable timestamp)
bulk_wave_dataframe['timestamp'] = pd.to_datetime(bulk_wave_dataframe['time'], unit="s", utc=True)

## Last Spotter observations
Plot last data point from every Spotter reported, and color by waveheight.

In [43]:
# Select last data available for each Spotter ID
last_data = bulk_wave_dataframe.drop_duplicates(subset="spotter_id", keep="last")


# set color thresholds
wave_min = 0  # m
wave_max = 8  # m

# make labels
marker_labels = last_data["spotter_id"] + ", Hs=" + last_data["significantWaveHeight"].round(decimals=1).astype(str) + " m"

# Use Plotly to visualize data
fig = go.Figure()
fig.add_trace(
    go.Scattergeo(
        lon=last_data["longitude"],
        lat=last_data["latitude"],
        marker=dict(
            color=last_data["significantWaveHeight"],
            colorscale="Spectral_r",
            size=10,
            cmin=wave_min,
            cmax=wave_max,
            colorbar={"title": "Observed Hs [m]"},
            line=dict(width=1, color="DarkSlateGrey"),
        ),
    text = marker_labels,
    hoverinfo = 'text'))

fig.update_geos(
    landcolor="#2B3746",
    lataxis_range=[-70, 70],
    lonaxis_range=[-180, 180],
)

fig.update_layout(title=f"Last Spotter observations")
fig.show()

## Plot trajectory of a single drifter

In [48]:
chosen_spotter_id = "SPOT-010323"
single_spotter_data_all = bulk_wave_dataframe.loc[bulk_wave_dataframe["spotter_id"]==chosen_spotter_id]
single_spotter_data = single_spotter_data_all.iloc[::10] # downsample by 10

# make labels (we don't need the ID b/c all one unit)
single_spotter_marker_labels =  single_spotter_data["timestamp"].astype(str) + "Hs=" + single_spotter_data["significantWaveHeight"].round(decimals=1).astype(str) + " m"

# Use Plotly to visualize data
fig = go.Figure()
fig.add_trace(
    go.Scattergeo(
        lon=single_spotter_data["longitude"],
        lat=single_spotter_data["latitude"],
        marker=dict(
            color=single_spotter_data["significantWaveHeight"],
            colorscale="Spectral_r",
            size=10,
            cmin=wave_min,
            cmax=wave_max,
            colorbar={"title": "Observed Hs [m]"},
        ),
    text = single_spotter_marker_labels,
    hoverinfo = 'text'))

fig.update_geos(
    landcolor="#2B3746",
    lataxis_range=[-70, 70],
    lonaxis_range=[-180, 180],
)

fig.update_layout(title=f"{chosen_spotter_id} observations")
fig.show()

## Plot timeseries of data from single Spotter

In [51]:
import plotly.express as px

fig = px.line(single_spotter_data_all, x='timestamp', y="significantWaveHeight",
              markers=True,
             labels={
                     "significantWaveHeight": "Significant Wave Height [m]",
                     "timestamp": "Date",
                 })

fig.show()

In [52]:
fig = px.line(single_spotter_data_all, x='timestamp', y="peakPeriod",
              markers=True,
             labels={
                     "peakPeriod": "Peak Period [s]",
                     "timestamp": "Date",
                 })

fig.show()

In [53]:
fig = px.line(single_spotter_data_all, x='timestamp', y="meanDirection",
              markers=True,
             labels={
                     "peakPeriod": "Mean Direction [degrees]",
                     "timestamp": "Date",
                 })

fig.show()